## Sticker info bot

In [ ]:
import os
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, ForceReply, Update
from telegram.ext import (
    Updater, 
    CommandHandler, 
    MessageHandler, 
    Filters, 
    CallbackContext, 
    CallbackQueryHandler, 
    ConversationHandler,
)

In [ ]:
# bot's token
TOKEN = os.getenv('TOKEN')

In [ ]:
class Sticker:

    # state for ConversationHandler
    DEFAULT = range(1)
    # callback data
    ID, SET, SIZE, ANIMATED, CANCEL = range(5)
    # sticker
    sticker = None

    def __init__(self, dispatcher) -> None:
        conv_handler = ConversationHandler(
            entry_points=[MessageHandler(Filters.sticker, self.received)],
            states={
                self.DEFAULT: [
                    CallbackQueryHandler(self.id, pattern='^' + str(self.ID) + '$'),
                    CallbackQueryHandler(self.set, pattern='^' + str(self.SET) + '$'),
                    CallbackQueryHandler(self.size, pattern='^' + str(self.SIZE) + '$'),
                    CallbackQueryHandler(self.animated, pattern='^' + str(self.ANIMATED) + '$'),
                ]
            },
            fallbacks=[CallbackQueryHandler(self.cancel, pattern='^' + str(self.CANCEL) + '$')],
        )
        dispatcher.add_handler(conv_handler)

    def received(self, update: Update, context: CallbackContext) -> int:
        self.sticker = update.message.sticker
        keyboard = [
            [
                InlineKeyboardButton("Sticker id", callback_data=str(self.ID)),
                InlineKeyboardButton("Sticker set", callback_data=str(self.SET)),
                InlineKeyboardButton("Sticker size", callback_data=str(self.SIZE)),
                InlineKeyboardButton("Animation", callback_data=str(self.ANIMATED)),
            ],
            [InlineKeyboardButton("Cancel", callback_data=str(self.CANCEL))],
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        update.message.reply_text(
            'Nice sticker! What do you want to do know about it?', 
            reply_markup=reply_markup
        )
        return self.DEFAULT
        
    def id(self, update: Update, context: CallbackContext) -> int:
        global sticker
        keyboard = [
            [
                InlineKeyboardButton("Sticker set", callback_data=str(self.SET)),
                InlineKeyboardButton("Sticker size", callback_data=str(self.SIZE)),
                InlineKeyboardButton("Animation", callback_data=str(self.ANIMATED)),
            ],
            [InlineKeyboardButton("Cancel", callback_data=str(self.CANCEL))],
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        query = update.callback_query
        query.answer()
        query.edit_message_text(
            text=f"Sticker's id: {self.sticker.file_id}\n\nDo you want to know anything else?",
            reply_markup=reply_markup
        )
        return self.DEFAULT

    def set(self, update: Update, context: CallbackContext) -> int:
        global sticker
        keyboard = [
            [
                InlineKeyboardButton("Sticker id", callback_data=str(self.ID)),
                InlineKeyboardButton("Sticker size", callback_data=str(self.SIZE)),
                InlineKeyboardButton("Animation", callback_data=str(self.ANIMATED)),
            ],
            [InlineKeyboardButton("Cancel", callback_data=str(self.CANCEL))],
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        query = update.callback_query
        query.answer()
        query.edit_message_text(
            text=f"Sticker set: {self.sticker.set_name}\n\nDo you want to know anything else?",
            reply_markup=reply_markup
        )
        return self.DEFAULT

    def size(self, update: Update, context: CallbackContext) -> int:
        global sticker
        keyboard = [
            [
                InlineKeyboardButton("Sticker id", callback_data=str(self.ID)),
                InlineKeyboardButton("Sticker set", callback_data=str(self.SET)),
                InlineKeyboardButton("Animation", callback_data=str(self.ANIMATED)),
            ],
            [InlineKeyboardButton("Cancel", callback_data=str(self.CANCEL))],
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        query = update.callback_query
        query.answer()
        query.edit_message_text(
            text=f"Sticker's size: {self.sticker.width}x{self.sticker.height}\n\nDo you want to know anything else?",
            reply_markup=reply_markup
        )
        return self.DEFAULT

    def animated(self, update: Update, context: CallbackContext) -> int:
        global sticker
        keyboard = [
            [
                InlineKeyboardButton("Sticker id", callback_data=str(self.ID)),
                InlineKeyboardButton("Sticker set", callback_data=str(self.SET)),
                InlineKeyboardButton("Sticker size", callback_data=str(self.SIZE)),
            ],
            [InlineKeyboardButton("Cancel", callback_data=str(self.CANCEL))],
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        query = update.callback_query
        query.answer()
        if self.sticker.is_animated:
            text = "Sticker is animated\n\nDo you want to know anything else?"
        else:
            text = "Sticker is not animated\n\nDo you want to know anything else?"
        query.edit_message_text(
            text=text,
            reply_markup=reply_markup
        )
        return self.DEFAULT

    def cancel(self, update: Update, context: CallbackContext) -> int:
        query = update.callback_query
        query.answer()
        query.edit_message_text(text="I'll be waiting for another sticker!")
        return ConversationHandler.END
    
    
def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text("Send sticker to get info")
    
def help(update: Update, context: CallbackContext) -> None:
    update.message.reply_text("Oops, can't help you")
    
def main() -> None:
    
    updater = Updater(TOKEN)
    dispatcher = updater.dispatcher

    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help))

    sticker = Sticker(dispatcher)
    
    updater.start_polling()
    updater.idle()

In [ ]:
main()